Импорт библиотек

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from xgboost import XGBRegressor

подключаем диск

In [2]:
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/MyDrive

Mounted at /content/drive/
/content/drive/MyDrive


Скачайте датасет, выберите столбец Price в качестве целевого, удалите его из X, затем сделайте сплит.

In [3]:
df = pd.read_csv('/content/drive/MyDrive/fourthsemestr/pipeline/Laptop_price.csv')
X = df.drop('Price', axis=1)
y = df['Price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Создайте переменные для категориальных и количественных типов данных и запишите в них данные.

In [4]:
categoricalFeatures = X.select_dtypes(include=['object']).columns.tolist()
numericalFeatures = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

Далее создайте пайплайн

In [5]:
numericalTransformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categoricalTransformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numericalTransformer, numericalFeatures),
        ('cat', categoricalTransformer, categoricalFeatures)
    ])
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', XGBRegressor(random_state=42))
])

В param_grid подберите оптимальные значения следующих гиперпараметров: n_estimators; learning_rate; max_depth; gamma; и один любой на выбор

In [8]:
param = {
    'model__n_estimators': [50, 100, 200],
    'model__learning_rate': [0.01, 0.1, 0.2],
    'model__max_depth': [3, 5, 7],
    'model__gamma': [0, 0.1, 0.2],
    'model__subsample': [0.8, 1.0]
}

In [9]:
gridSearch = GridSearchCV(pipeline, param, cv=5, n_jobs=-1, verbose=2)
gridSearch.fit(X_train, y_train)
model = gridSearch.best_estimator_
bestParams = gridSearch.best_params_
print("Лучшие параметры: ", bestParams)

Fitting 5 folds for each of 162 candidates, totalling 810 fits
Лучшие параметры:  {'model__gamma': 0, 'model__learning_rate': 0.1, 'model__max_depth': 3, 'model__n_estimators': 100, 'model__subsample': 1.0}
